### - Sobhan Moradian Daghigh
#### - 7/10/2022
#### - Convex - HW04 - Q1

In [1]:
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split

from joblib import Parallel, delayed
import multiprocessing

### Reading Dataset

In [2]:
dataset = pd.read_excel('./ENB2012_data.xlsx')
dataset.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    int64  
 6   X7      768 non-null    float64
 7   X8      768 non-null    int64  
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 60.1 KB


### Normalization

In [4]:
normal = preprocessing.normalize(dataset)
dataset = pd.DataFrame(normal, columns=dataset.columns)
dataset.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.001624,0.852714,0.487265,0.182725,0.011602,0.003315,0.0,0.0,0.025772,0.035352
1,0.001624,0.852709,0.487262,0.182723,0.011601,0.004972,0.0,0.0,0.025772,0.035351
2,0.001624,0.852700,0.487257,0.182722,0.011601,0.006629,0.0,0.0,0.025772,0.035351
3,0.001624,0.852690,0.487251,0.182719,0.011601,0.008287,0.0,0.0,0.025771,0.035351
4,0.001364,0.854113,0.482759,0.185677,0.010610,0.003031,0.0,0.0,0.031588,0.042865


### Split dataset into 80% of train and 20% of test

In [5]:
X = dataset.iloc[:, :-2]
y = dataset.iloc[:, -2:]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [6]:
x_train.shape, x_test.shape

((614, 8), (154, 8))

### Some functions:

In [7]:
def next_x(A, b, r, z, v, lenght):
    return (np.linalg.inv(A.T.dot(A) + (r * np.identity(lenght)))).dot(A.T.dot(b) + (r * z) - v)

In [13]:
def next_z(z, x, v, r, lambda_):
    part1 = x + np.divide(v, r)
    
    # sign(z) ---------------
    less    = -np.divide(lambda_, r) * (z < 0)
    greater =  np.divide(lambda_, r) * (z > 0)
    return part1 - less - greater

In [9]:
def next_v(v, r, nxt_x, nxt_z):
    return v + (r * (nxt_x - nxt_z)) 

In [10]:
def get_regression(A, x, b, lambda_):
    part1 = 0.5 * np.linalg.norm(A.dot(x) - b, ord='fro') ** 2
    part2 = lambda_ * np.linalg.norm(x, ord=1)
    return part1 + part2

In [19]:
def admm(A_train, b_train, A_test, b_test, iters=100, lambda_=0.01):
    feature_lenght = A_train.shape[1]
    r = 1
    nxt_z = np.zeros((feature_lenght, 1))
    nxt_v = np.zeros((feature_lenght, 1))
    nxt_x = np.random.randn(feature_lenght, 1)
    
    for iteration in range(iters):
        nxt_x = next_x(A=A_train, b=b_train, r=r, z=nxt_z, v=nxt_v, lenght=feature_lenght)
        nxt_z = next_z(z=nxt_z, x=nxt_x, v=nxt_v, r=r, lambda_=lambda_)
        nxt_v = next_v(v=nxt_v, r=r, nxt_x=nxt_x, nxt_z=nxt_z)
        
        result = get_regression(A=A_train, x=nxt_x, b=b_train, lambda_=lambda_)
        train_rmse = mean_squared_error(b_train, A_train.dot(nxt_x), squared=False)
        test_rmse  = mean_squared_error(b_test , A_test.dot(nxt_x) , squared=False)
        print('iter: {:<7} result: {:<12.5f} TrainRMSE: {:<12.5f} TestRMSE: {:<12.5f}'.format(iteration, result, train_rmse, test_rmse))    

### Heating Load: Y1


In [65]:
b_train = np.array(y_train.iloc[:, 0]).reshape(-1, 1)
b_test  = np.array( y_test.iloc[:, 0]).reshape(-1, 1)
admm(A_train=x_train, b_train=b_train, A_test=x_test, b_test=b_test, iters=100, lambda_=0.001)

iter: 0       result: 0.02137      TrainRMSE: 0.00830      TestRMSE: 0.00871     
iter: 1       result: 0.01889      TrainRMSE: 0.00779      TestRMSE: 0.00825     
iter: 2       result: 0.01876      TrainRMSE: 0.00776      TestRMSE: 0.00823     
iter: 3       result: 0.01873      TrainRMSE: 0.00775      TestRMSE: 0.00823     
iter: 4       result: 0.01871      TrainRMSE: 0.00774      TestRMSE: 0.00822     
iter: 5       result: 0.01869      TrainRMSE: 0.00774      TestRMSE: 0.00822     
iter: 6       result: 0.01866      TrainRMSE: 0.00773      TestRMSE: 0.00821     
iter: 7       result: 0.01864      TrainRMSE: 0.00773      TestRMSE: 0.00821     
iter: 8       result: 0.01862      TrainRMSE: 0.00772      TestRMSE: 0.00820     
iter: 9       result: 0.01860      TrainRMSE: 0.00771      TestRMSE: 0.00819     
iter: 10      result: 0.01857      TrainRMSE: 0.00771      TestRMSE: 0.00819     
iter: 11      result: 0.01855      TrainRMSE: 0.00770      TestRMSE: 0.00818     
iter: 12      re

### Cooling Load: Y2

In [66]:
b_train = np.array(y_train.iloc[:, 1]).reshape(-1, 1)
b_test  = np.array( y_test.iloc[:, 1]).reshape(-1, 1)
admm(A_train=x_train, b_train=b_train, A_test=x_test, b_test=b_test, iters=100, lambda_=0.001)

iter: 0       result: 0.02215      TrainRMSE: 0.00845      TestRMSE: 0.00834     
iter: 1       result: 0.01986      TrainRMSE: 0.00799      TestRMSE: 0.00793     
iter: 2       result: 0.01974      TrainRMSE: 0.00796      TestRMSE: 0.00792     
iter: 3       result: 0.01971      TrainRMSE: 0.00795      TestRMSE: 0.00792     
iter: 4       result: 0.01968      TrainRMSE: 0.00795      TestRMSE: 0.00791     
iter: 5       result: 0.01965      TrainRMSE: 0.00794      TestRMSE: 0.00790     
iter: 6       result: 0.01963      TrainRMSE: 0.00793      TestRMSE: 0.00790     
iter: 7       result: 0.01960      TrainRMSE: 0.00793      TestRMSE: 0.00789     
iter: 8       result: 0.01957      TrainRMSE: 0.00792      TestRMSE: 0.00788     
iter: 9       result: 0.01954      TrainRMSE: 0.00791      TestRMSE: 0.00788     
iter: 10      result: 0.01952      TrainRMSE: 0.00791      TestRMSE: 0.00787     
iter: 11      result: 0.01949      TrainRMSE: 0.00790      TestRMSE: 0.00787     
iter: 12      re

### Parallel ADMM

In [45]:
def next_v_parallel(nxt_v, nxt_x, nxt_z, r):
    return np.array([(nxt_v[i].reshape(-1, 1) + r * (nxt_x[i].reshape(-1, 1) - nxt_z)).T for i in range(nxt_v.shape[0])])

In [42]:
def parallel_process(i, A, b, nxt_v, r, nxt_z, nxt_x):
    idx_a = np.array(A.iloc[i, :]).reshape(-1, 1).T
    idx_b = (b[i].reshape(-1, 1)).item()

    idx_v = nxt_v[i].reshape(-1, 1)
    term1 = (1 / (idx_a.dot(idx_a.T) + r)).item()
    term2 = idx_a.T * idx_b + r * nxt_z - idx_v 

    nxt_x[i] = (term1 * term2).T

In [43]:
def parallel_admm(A_train, b_train, A_test, b_test, iters=100, lambda_=0.01):
    data_lenght, feature_lenght = A_train.shape
    r = 1
    nxt_z = np.zeros((feature_lenght, 1))
    nxt_v = np.tile(np.zeros((feature_lenght, 1)).T, (data_lenght, 1))
    nxt_x = np.tile(np.random.randn(feature_lenght, 1).T, (data_lenght, 1))
        
    for iteration in range(iters):
        Parallel(n_jobs=multiprocessing.cpu_count(), require='sharedmem')(delayed(parallel_process)(i, A_train, b_train, nxt_v, r, nxt_z, nxt_x) for i in range(data_lenght))
        x_mean = np.mean(nxt_x, axis=0).reshape(-1, 1)
        v_mean = np.mean(nxt_v, axis=0).reshape(-1, 1)
        
        nxt_z = next_z(z=nxt_z, x=x_mean, v=v_mean, r=r, lambda_=lambda_)
        
        nxt_v = next_v_parallel(nxt_v=nxt_v, nxt_x=nxt_x, nxt_z=nxt_z, r=r)
        
        result = get_regression(A=A_train, x=x_mean, b=b_train, lambda_=lambda_)
        train_rmse = mean_squared_error(b_train, A_train.dot(x_mean), squared=False)
        test_rmse  = mean_squared_error(b_test , A_test.dot(x_mean) , squared=False)
        print('iter: {:<7} result: {:<12.5f} TrainRMSE: {:<12.5f} TestRMSE: {:<12.5f}'.format(iteration, result, train_rmse, test_rmse))    

### Heating Load: Y1

In [67]:
b_train = np.array(y_train.iloc[:, 0]).reshape(-1, 1)
b_test  = np.array( y_test.iloc[:, 0]).reshape(-1, 1)
parallel_admm(A_train=x_train, b_train=b_train, A_test=x_test, b_test=b_test, iters=100, lambda_=0.001)

iter: 0       result: 0.14376      TrainRMSE: 0.02164      TestRMSE: 0.02277     
iter: 1       result: 0.09109      TrainRMSE: 0.01722      TestRMSE: 0.01817     
iter: 2       result: 0.08216      TrainRMSE: 0.01636      TestRMSE: 0.01721     
iter: 3       result: 0.07676      TrainRMSE: 0.01581      TestRMSE: 0.01657     
iter: 4       result: 0.07509      TrainRMSE: 0.01563      TestRMSE: 0.01634     
iter: 5       result: 0.07427      TrainRMSE: 0.01555      TestRMSE: 0.01622     
iter: 6       result: 0.07408      TrainRMSE: 0.01553      TestRMSE: 0.01619     
iter: 7       result: 0.07383      TrainRMSE: 0.01550      TestRMSE: 0.01616     
iter: 8       result: 0.07388      TrainRMSE: 0.01551      TestRMSE: 0.01616     
iter: 9       result: 0.07373      TrainRMSE: 0.01549      TestRMSE: 0.01614     
iter: 10      result: 0.07383      TrainRMSE: 0.01550      TestRMSE: 0.01615     
iter: 11      result: 0.07371      TrainRMSE: 0.01549      TestRMSE: 0.01614     
iter: 12      re

### Cooling Load: Y2

In [68]:
b_train = np.array(y_train.iloc[:, 1]).reshape(-1, 1)
b_test  = np.array( y_test.iloc[:, 1]).reshape(-1, 1)
parallel_admm(A_train=x_train, b_train=b_train, A_test=x_test, b_test=b_test, iters=100, lambda_=0.001)

iter: 0       result: 0.15539      TrainRMSE: 0.02250      TestRMSE: 0.02316     
iter: 1       result: 0.09153      TrainRMSE: 0.01726      TestRMSE: 0.01776     
iter: 2       result: 0.08013      TrainRMSE: 0.01615      TestRMSE: 0.01656     
iter: 3       result: 0.07382      TrainRMSE: 0.01550      TestRMSE: 0.01582     
iter: 4       result: 0.07187      TrainRMSE: 0.01530      TestRMSE: 0.01557     
iter: 5       result: 0.07095      TrainRMSE: 0.01520      TestRMSE: 0.01545     
iter: 6       result: 0.07072      TrainRMSE: 0.01517      TestRMSE: 0.01541     
iter: 7       result: 0.07046      TrainRMSE: 0.01514      TestRMSE: 0.01538     
iter: 8       result: 0.07049      TrainRMSE: 0.01515      TestRMSE: 0.01538     
iter: 9       result: 0.07035      TrainRMSE: 0.01513      TestRMSE: 0.01536     
iter: 10      result: 0.07044      TrainRMSE: 0.01514      TestRMSE: 0.01537     
iter: 11      result: 0.07032      TrainRMSE: 0.01513      TestRMSE: 0.01536     
iter: 12      re

### LassoCV

In [49]:
def lasso_regression(A_train, b_train, A_test, b_test):
    model = LassoCV(cv=5, random_state=0).fit(A_train, b_train)
    predict_train = model.predict(A_train)
    predict_test = model.predict(A_test)
    
    train_rmse = mean_squared_error(b_train, predict_train, squared=False)
    test_rmse  = mean_squared_error(b_test , predict_test , squared=False)
    print('TrainRMSE: {:<12.5f} TestRMSE: {:<12.5f}'.format(train_rmse, test_rmse))    

### Heating Load: Y1

In [54]:
b_train = y_train.iloc[:, 0]
b_test  = y_test.iloc[:, 0]
lasso_regression(A_train=x_train, b_train=b_train, A_test=x_test, b_test=b_test)

TrainRMSE: 0.00546      TestRMSE: 0.00626     


### Cooling Load: Y2

In [55]:
b_train = y_train.iloc[:, 1]
b_test  = y_test.iloc[:, 1]
lasso_regression(A_train=x_train, b_train=b_train, A_test=x_test, b_test=b_test)

TrainRMSE: 0.00532      TestRMSE: 0.00549     


### Finito